In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
np.set_printoptions(precision=4)

In [ ]:
# g === area (big image)
# f === fragment
g = Image.open('img/20160825-143207-156-1.jpg').convert('L')

wg = g.width // 6
hg = g.height // 6
xg = 200
yg = 200

box = (xg, yg, xg + wg, yg + hg)
g = g.crop(box)
g

In [ ]:
print g.size
g_mat = np.asarray(g.getdata(), dtype=int).reshape(g.size[1], g.size[0])

In [ ]:
# Crop fragment from area image
wf = 100
hf = 100
# For moving averages computing
offset_x = 20
offset_y = 20
xf = np.random.randint(offset_x, g.size[0] - wf - offset_x)
yf = np.random.randint(offset_y, g.size[1] - hf - offset_y)


crop_box = (xf, yf, xf + wf, yf + hf)
f = g.crop(crop_box)
print f.size
print yf, xf
f

In [ ]:
# Create matrix of fragment
f_mat = np.asarray(f.getdata(), dtype=np.int).reshape(f.size[1], f.size[0])
# Flip matrix
f_mat = np.fliplr(f_mat)
f_mat = np.flipud(f_mat)

Image.fromarray(f_mat.astype('uint8'), 'L')

In [ ]:
num_shades = 256
# Create indicators of f
# of size == g.size
chi = np.zeros((num_shades, g.size[1], g.size[0]), dtype=bool)

In [ ]:
# fill the indicators
for h in xrange(f.size[1]):
    for w in xrange(f.size[0]):
        color = f_mat[h, w]
        chi[color, h, w] = True

In [ ]:
Image.fromarray(chi.sum(axis=0).astype('uint8')*(num_shades - 1), 'L')

In [ ]:
# chi_elems[i] === number of pixels that have color "i"
chi_elems = np.array( f.histogram() )
print chi_elems

In [ ]:
fft_chi = np.fft.fft2(chi)

In [ ]:
fft_g = np.fft.fft2(g_mat)

In [ ]:
# Scalar product (g_frag, chi[i])
sp_g_frag_chi = np.zeros((num_shades, g.size[1] - hf, g.size[0] - wf))

for i in xrange(num_shades):
    if chi_elems[i] > 0:
        sp_g_frag_chi[i] = np.fft.ifft2(fft_g * fft_chi[i])[hf:, wf:]

In [ ]:
# || Projection of g_frag on f ||^2
norm_pr_gfrag_sqr = np.zeros((g.size[1] - hf, g.size[0] - wf))
for i in xrange(num_shades):
    if chi_elems[i] > 0:
        norm_pr_gfrag_sqr += sp_g_frag_chi[i] ** 2 / float(chi_elems[i])
        

In [ ]:
plt.plot(norm_pr_gfrag_sqr.ravel())

In [ ]:
# chi_X -- const field of vision
# 1 1 1 0 0 ... 0
# 1 1 1 0 0 ... 0
# 1 1 1 0 0 ... 0
# 0 0 0 0 0 ... 0
# . . .
# 0 0 0 0 0 ... 0
chi_X = np.zeros((g.size[1], g.size[0]), dtype=bool)
chi_X[:hf, :wf] = np.ones((hf, wf))

print g_mat.min()
print (g_mat**2).min()

# || g ||^2
fft_gsqr = np.fft.fft2(g_mat ** 2)
fft_chi_X = np.fft.fft2(chi_X)
norm_gfrag_sqr = np.fft.ifft2(fft_gsqr * fft_chi_X)[hf:, wf:].astype('float')

# norm_gfrag_sqr = abs(norm_gfrag_sqr)

In [ ]:
plt.plot(norm_gfrag_sqr.ravel())

In [ ]:
# E_gfrag = np.fft.ifft2(fft_g * fft_chi_X)[hf:, wf:].astype('float')
norm_E_gfrag_sqr = np.fft.ifft2(fft_g * fft_chi_X)[hf:, wf:].astype('float') \
                        ** 2 / (hf * wf)

In [ ]:
plt.plot(norm_E_gfrag_sqr.ravel())

In [ ]:
numerator = norm_gfrag_sqr - norm_pr_gfrag_sqr
plt.plot(numerator.ravel())
print numerator.min()

In [ ]:
denominator = norm_pr_gfrag_sqr - norm_E_gfrag_sqr
plt.plot(denominator.ravel())

In [ ]:
tau = abs(numerator) / abs(denominator)
plt.plot(tau.ravel(), label='tau')
plt.legend()

In [ ]:
index = tau.argmin() 
x_min = index % tau.shape[1] + 1
y_min = index // tau.shape[1] + 1
print index
print 'Should be:', true_idx
print "y_min, x_min: %d %d" % (y_min, x_min)
print 'Should be:', (yf, xf)

In [ ]:
Image.fromarray(g_mat[y_min : y_min + hf , x_min : x_min + wf].astype('uint8'), 'L')

In [ ]:
f